In [1]:
import os
from openai import OpenAI
import pandas as pd
import spacy
# import requests
# from spacy.matcher import Matcher
# from spacy import displacy
# from transformers import pipeline
# from transformers import T5ForConditionalGeneration, T5Tokenizer
from geopy.geocoders import Nominatim
import re

In [2]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY") or "",
) 
noticias_agosto = 'test_data2.csv'  
df = pd.read_csv(noticias_agosto, encoding='utf-8')
nlp = spacy.load("es_core_news_sm")


In [3]:
def crear_prompt(text):
    """Crear el prompt para el modelo de chat."""
    return (
        f"Hola, te entregaré el siguiente fragmento de una noticia y necesito que me "
        f"entregues el evento principal , la ubicación y la categoria, la categoria tiene que estar estrictamente en este formato: "
        f"sociedad, salud, politica, medioambiente, internacional, entretenimiento, economia, deportes, cultura, cienciatecnologia"
        f"Clasificala solo con una categoria."
        f" Estas dámelas separadas con un /, es decir Evento: .... / Ubicación: .... / Categoria: ...."
        f"La ubicación debe estar en el "
        f"siguiente formato y contener tantos detalles como sea posible: \n\n"
        f"Lugar, Ciudad, País"
        f"Si no encuentras ciudad o lugar, solo coloca el país; si no encuentras ninguno de esos 3 coloca Chile solamente."
        f"No utilices conectores ni palabras como población, localidad, comunidad, etc..."
        f"Este es el texto: \"{text}\". "
        f"Por favor, dame la respuesta siguiendo estrictamente este formato."
    )

In [4]:
def obtener_respuesta(prompt, client):
    """Obtener la respuesta del modelo de chat."""
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt 
            }
        ],
        model="gpt-3.5-turbo",
    )
    return chat_completion.choices[0].message.content

In [5]:
palabras_no_deseadas = ["villa", "población", "comuna", "sector", "avenida", "calle", "pasaje"]

def extraer_nombres_propios_sin_adp(texto):
    doc = nlp(texto)

    # Filtrar nombres propios (PROPN) y eliminar las preposiciones (ADP)
    nombres_limpios = [token.text for token in doc if token.pos_ == "PROPN" and token.text.lower() not in palabras_no_deseadas]
    
    return " ".join(nombres_limpios)


In [6]:
def procesar_mensaje(mensaje):
    lineas = mensaje.split("/")
    
    if len(lineas) < 2:  # Verificar que haya al menos 2 líneas (evento y ubicación)
        print(f"Formato inesperado en el mensaje: {mensaje}")
        return None, None, None  # O lanza una excepción, dependiendo de tu lógica
    
    # Asegurarte de que cada línea tenga el formato esperado
    evento = lineas[0].split(": ")[1] if len(lineas[0].split(": ")) > 1 else "No disponible"
    ubicacion = lineas[1].split(": ")[1] if len(lineas[1].split(": ")) > 1 else "No disponible"
    categoria = lineas[2].split(": ")[1] if len(lineas) > 2 and len(lineas[2].split(": ")) > 1 else "No disponible"

    return evento, ubicacion, categoria
    

In [13]:
def cordenadas(ubicacion):
    """Obtener las coordenadas de la ubicación usando Nominatim."""
    geolocator = Nominatim(user_agent="Sophia", timeout=10)  
    location = geolocator.geocode(ubicacion)
    print(location)
    if location:
        return location.address, location.latitude, location.longitude
    else:
        return None, None, None



##### Obtener evento de la noticia

Para este enfoque se utilizo openai (aunque antes se utilizo matcher y transformers tambien, los cuales en algunos casos se comportaban bien, pero cuando la noticia cambiaba mucho o estaba escrita de diferentes formas la calidad de la respuesta decaia), la idea es preguntar a openai y aprovechar sus funcionalidades al maximo para obtener una respuesta de buena calidad.

In [8]:
resultados = []

for index in range(len(df)):
    text = df.iloc[index]['text']
    if text:
        prompt = crear_prompt(text)
        mensaje = obtener_respuesta(prompt, client)
        evento, ubicacion, categoria = procesar_mensaje(mensaje)
        direccion_formateada = extraer_nombres_propios_sin_adp(ubicacion)
        direccion, latitud, longitud = cordenadas(direccion_formateada)
       
            # Guardar los resultados en una lista
        resultados.append({
                'id_news': df.iloc[index]['id_news'],  
                'event': evento,
                'category': categoria,
                'address': direccion,
                'latitud': latitud,
                'longitud': longitud
            })

    # Crear un DataFrame a partir de los resultados
resultados_df = pd.DataFrame(resultados)
resultados_df.to_csv("output.csv", index=False, encoding='utf-8-sig')



Mensaje recibido: Evento: Balacera en Quilicura deja un muerto y dos heridos / Ubicación: San Ignacio con Cerro Los Cóndores, Quilicura, Chile / Categoria: sociedad
Mensaje recibido: Evento: Protesta de vecinos en Población Manuel Rodríguez / Ubicación: La Calera, Chile / Categoria: sociedad
Mensaje recibido: Evento:Festival Nacional de Cine de Estudiantes Secundarios (FESCIES) 2024 / Ubicación:Valparaíso, Chile / Categoria:cultura
Mensaje recibido: Evento: Masivo operativo de la PDI y Carabineros en La Pintana / Ubicación: Villa La Orquesta, La Pintana, Chile / Categoria: sociedad
Mensaje recibido: Evento: Anuncio de lista de interesados para explotar litio en Chile / Ubicación: Chile / Categoria: economia
Mensaje recibido: Evento: Operativo por violencia intrafamiliar (VIF) en La Florida / Ubicación: Los Quillayes, La Florida, Chile / Categoria: sociedad
Mensaje recibido: Evento: Retroceso del dólar en mercado cambiario local / Ubicación: Chile / Categoria: economía
Mensaje recibido: